In [1]:
# # the dependencies here are a mess, I would love to have these all in the environment.yml file, but I don't know how to pass the "--no-deps" flag to pip install from the environment.yml file.
# !pip uninstall -y pangeo-forge-runner pangeo-forge-recipes
# !pip install git+https://github.com/pangeo-forge/pangeo-forge-runner.git@arbitrary-injections --no-deps
# !pip install git+https://github.com/jbusecke/pangeo-forge-recipes.git@dynamic_chunks_2 --no-deps
# !pip install ruamel.yaml --no-deps


In [3]:
# # test bigquery logic (WTF? Why is this working in the other notebook but not here)
# from feedstock.recipe import urls_from_bq
# urls_from_bq('CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r3i1p1f1.day.pr.gn.v20190603')

In [4]:
pip list | grep pangeo-forge

pangeo-forge-esgf               0.0.5.dev16+g172f1d0
pangeo-forge-recipes            0.6.2.dev549+g91eb6a6
pangeo-forge-runner             0.7.2
Note: you may need to restart the kernel to use updated packages.


In [5]:
# global input args
repo = "https://github.com/jbusecke/cmip6-leap-feedstock.git"
ref = "beam-dataflow-testing"
# f_name = "config.json"
# test locally 
f_name = "config_local.json"

In [6]:
def shorten_id(iid):
    iid_stripped = ''.join(iid.lower().split('.')[3:])
    assert len(iid_stripped)<66
    return iid_stripped

In [7]:
# from the integration tests of pangeo-forge-runner
#https://github.com/pangeo-forge/pangeo-forge-runner/blob/main/tests/integration/test_dataflow_integration.py

import subprocess
import os
import time
import xarray as xr
import json
import random
import string

# using randomchoice due to this https://stackoverflow.com/questions/13484726/safe-enough-8-character-short-unique-random-string
alphabet = string.ascii_lowercase + string.digits

# # check written stores (should this just be
# iids = []
# with open('iids.txt') as f:
#     for line in f:
#         if not '#' in line:
#             iids.append(str(line).replace(" ", "").replace("',\n", "").replace("'","")) # FIXME: This seems ugly haha
# print(iids)

# iids = [
#     'CMIP6.CMIP.IPSL.IPSL-CM6A-LR.historical.r1i1p1f1.Omon.zmeso.gn.v20180803',
#     # 'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp585.r5i1p1f1.day.sfcWind.gn.v20210907'
# ]

# tims full request (minus the miroc ones that were not available

# ec-earth3ssp585r140i1p1f1daypslgrv2020041

iids = [
#     'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r17i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r2i3p1f1.day.pr.gn.v20201222',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r25i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r127i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r138i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r15i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r134i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r110i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r1i3p1f1.day.pr.gn.v20201222',
#  'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r4i3p1f1.day.pr.gn.v20201222',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r27i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r103i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r1i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r142i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r124i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r29i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r7i1p1f2.day.pr.gr.v20210218',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r2i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r4i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r15i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r111i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r126i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r136i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r20i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r122i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r131i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r7i1p1f2.day.sfcWind.gr.v20210218',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r24i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r1i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r133i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r13i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r141i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r19i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r30i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r18i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r123i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r8i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r121i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r23i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r146i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r16i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r2i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r149i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r23i1p1f1.day.psl.gr.v20200407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r145i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r29i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.MOHC.HadGEM3-GC31-MM.ssp585.r2i1p1f3.day.psl.gn.v20200515',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r12i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r23i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r7i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r7i1p1f2.day.psl.gr.v20210218',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r14i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r132i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r9i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r118i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r21i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r13i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r115i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r26i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r129i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r6i1p1f2.day.sfcWind.gr.v20210517',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r12i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r113i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r106i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r15i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r12i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r114i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r6i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r25i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r128i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r12i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r17i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r147i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r17i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r27i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r4i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r21i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r101i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r130i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r2i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r150i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r19i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r17i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r108i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r144i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r21i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r25i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r10i1p1f2.day.psl.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r120i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r19i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r5i3p1f1.day.pr.gn.v20201222',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r5i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r125i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r18i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r29i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r28i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r10i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r1i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r112i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r8i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r30i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r30i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r10i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r143i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r139i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r10i1p1f2.day.sfcWind.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r8i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r137i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r19i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r23i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r140i1p1f1.day.psl.gr.v20200412', # fails with depenency issues? (only intermittent)
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r148i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r27i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r119i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r105i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r8i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r116i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r11i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r15i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r20i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r22i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r117i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r107i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r3i3p1f1.day.pr.gn.v20201222',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r16i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r4i1p1f1.day.pr.gn.v20190603',
#  'CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r5i1p1f1.day.pr.gn.v20190603',
#  'CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r2i1p1f1.day.pr.gn.v20190603',
 'CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r3i1p1f1.day.pr.gn.v20190603',
       ]


# iids = [
#     # 'CMIP6.ScenarioMIP.MOHC.HadGEM3-GC31-MM.ssp585.r2i1p1f3.day.psl.gn.v20200515',
#     # 'CMIP6.ScenarioMIP.MOHC.HadGEM3-GC31-MM.ssp585.r3i1p1f3.day.psl.gn.v20200507',
#     # 'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r140i1p1f1.day.psl.gr.v20200412', # fails with depenency issues?
#     # 'CMIP6.CMIP.IPSL.IPSL-CM5A2-INCA.historical.r1i1p1f1.Omon.zmeso.gn.v20200729',
# ]

# ## Testing 'throwing mem at the problem'
# high_mem_iids = [
#     'CMIP6.CMIP.NCC.NorESM2-LM.historical.r1i1p1f1.Omon.umo.gn.v20190815',
#     'CMIP6.CMIP.IPSL.IPSL-CM6A-LR.historical.r1i1p1f1.Omon.zmeso.gn.v20180803',
# ]
# iids = high_mem_iids
# # I know these failed with high mem use :https://console.cloud.google.com/dataflow/jobs/us-central1/2023-03-29_16_00_29-11923246449479218691;graphView=0?project=leap-pangeo&authuser=2&pageState=(%22dfTime%22:(%22l%22:%22dfJobMaxTime%22))
# # and https://console.cloud.google.com/dataflow/jobs/us-central1/2023-03-29_16_29_49-6440994980175925686;graphView=0?project=leap-pangeo&authuser=2&pageState=(%22dfTime%22:(%22l%22:%22dfJobMaxTime%22))
# # lets see if these beast machines can solve this issue?
# # maybe Ryans PR will alleviate this? It seems to do!

my_env = os.environ.copy()
my_env["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/juliusbusecke/Downloads/keys/leap-pangeo-dataflow_service_account.json"

def submit_job(repo, ref, f_name, iid, jobname, machine_type):
    inject_dict = {
        "get_iid": {"iid":iid}, 
        "get_jobname":{"jobname": jobname}
    }

    cmd = [
        "pangeo-forge-runner",
        "bake",
        "--repo",
        repo,
        "--ref",
        ref,
        "--json",
        "-f",
        f_name,
        "--Bake.inject",
        json.dumps(inject_dict),
        "--Bake.job_name",
        jobname,
        "--DataflowBakery.machine_type", # i hope this is just ignored in the local case?
        machine_type,
    ]

    print("\nSubmitting job...")
    print(f"{' '.join(cmd)}")
    submit_proc = subprocess.run(cmd, capture_output=True, env=my_env)
    return submit_proc



for iid in iids:
    print(f"""Submitting ({iid})""")
    unique_iid = iid+'.'+''.join(random.choices(alphabet, k=8))
    jobname=shorten_id(unique_iid)
    # machine_type = "e2-highmem-16" # Ill have to experiment with that
    # machine_type = "e2-highmem-8" # seems i had some other errors here...
    # As far as I can see in the recent 'job metrics', we never go above 1 core. So we only need the beast machines for the larger models
    # machine_type = "e2-standard-2"
    machine_type = "n2-highmem-2"
    # if iid in high_mem_iids:
    #     machine_type = "n2-highmem-16"
    # there are definitely some jobs failing with high mem useage. TODO: I need a way to override this for some of them...
    
    try:
        submit_proc = submit_job(repo, ref, f_name, iid, jobname, machine_type)
        assert submit_proc.returncode == 0
        lastline = json.loads(submit_proc.stdout.decode().splitlines()[-1])
        assert lastline["status"] == "submitted"
        job_id = lastline["job_id"]
        job_name = lastline["job_name"]
        
        print(f"Job submitted with {job_id = }")
        with open('submitted_iids.txt', 'a') as f:
            f.write(iid+'\n')
    except Exception as err:
        print(f'Submission FAILED with {err}')
        print('STDOUT')
        for line in submit_proc.stdout.decode().splitlines():
            print(line)
        print('STDERR')
        for line in submit_proc.stderr.decode().splitlines():
            print(line)

Submitting (CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r3i1p1f1.day.pr.gn.v20190603)

Submitting job...
pangeo-forge-runner bake --repo https://github.com/jbusecke/cmip6-leap-feedstock.git --ref beam-dataflow-testing --json -f config_local.json --Bake.inject {"get_iid": {"iid": "CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r3i1p1f1.day.pr.gn.v20190603"}, "get_jobname": {"jobname": "mri-esm2-0historicalr3i1p1f1dayprgnv20190603qwndu53a"}} --Bake.job_name mri-esm2-0historicalr3i1p1f1dayprgnv20190603qwndu53a --DataflowBakery.machine_type n2-highmem-2
Submission FAILED with 
STDOUT
{"message": "Target Storage is FSSpecTarget(LocalFileSystem(, root_path=\"local_storage/cache/{job_name}\")\n", "status": "setup"}
{"message": "Input Cache Storage is CacheFSSpecTarget(LocalFileSystem(, root_path=\"local_storage/cache/{job_name}\")\n", "status": "setup"}
{"message": "Metadata Cache Storage is MetadataTarget(AbstractFileSystem(, root_path=\"\")\n", "status": "setup"}
{"message": "Picked Git content provider.\n"

In [11]:
for line in submit_proc.stderr.decode().splitlines():
    print(line)

[Bake] WARNING | Config option `inject` not recognized by `Bake`.
[Bake] WARNING | Config option `inject` not recognized by `Bake`.


In [10]:
line

'[Bake] WARNING | Config option `inject` not recognized by `Bake`.'

In [7]:
submit_proc.stdout.decode().splitlines()

['{"message": "Target Storage is FSSpecTarget(LocalFileSystem(, root_path=\\"local_storage/cache/{job_name}\\")\\n", "status": "setup"}',
 '{"message": "Input Cache Storage is CacheFSSpecTarget(LocalFileSystem(, root_path=\\"local_storage/cache/{job_name}\\")\\n", "status": "setup"}',
 '{"message": "Metadata Cache Storage is MetadataTarget(AbstractFileSystem(, root_path=\\"\\")\\n", "status": "setup"}',
 '{"message": "Picked Git content provider.\\n", "status": "fetching"}',
 '{"message": "Cloning into \'/var/folders/_1/1k9jtjl51z333f21s7yht0340000gn/T/tmpngmbbcga\'...\\n", "status": "fetching"}',
 '{"message": "HEAD is now at c83067c Remove jobname injection\\n", "status": "fetching"}',
 '{"message": "Parsing recipes...", "status": "running"}',
 '{"message": "Fetching urls from GCS for None", "status": "running"}',
 '{"message": "\\n", "status": "running"}',
 '{"message": "Error during running: b/leap-persistent/o/jbusecke%2Fcmip6urls%2FNone.json", "exc_info": "Traceback (most recent 

In [6]:
submit_proc = submit_job(repo, ref, f_name, iid, jobname, machine_type)


Submitting job...


In [7]:
submit_proc

CompletedProcess(args=['pangeo-forge-runner', 'bake', '--repo', 'https://github.com/jbusecke/cmip6-leap-feedstock.git', '--ref', 'beam-dataflow-testing', '--json', '-f', 'config_local.json', '--Bake.inject', '{"get_iid": {"iid": "CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r3i1p1f1.day.pr.gn.v20190603"}, "get_jobname": {"jobname": "mri-esm2-0historicalr3i1p1f1dayprgnv20190603p9q2ookx"}}', '--Bake.job_name', 'mri-esm2-0historicalr3i1p1f1dayprgnv20190603p9q2ookx', '--DataflowBakery.machine_type', 'n2-highmem-2'], returncode=1, stdout=b'', stderr=b'Traceback (most recent call last):\n  File "/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/bin/pangeo-forge-runner", line 5, in <module>\n    from pangeo_forge_runner.cli import main\n  File "/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_runner/cli.py", line 3, in <module>\n    from .commands.bake import Bake\n  File "/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/si

In [6]:
len(iids)

129

In [7]:
# for line in submit_proc.stderr.decode().splitlines():
#     # print(json.loads(line)['message'])
#     print(line)

In [8]:
# err